In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import date, timedelta
import calendar

pd.options.display.max_rows = 30

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

In [2]:
name = 'TMT'
new_grade = 'C2'
new_period = '1'

sqlUpd = """
UPDATE buy
SET grade = "%s", period = "%s" WHERE name = "%s"
"""
sqlUpd = sqlUpd % (new_grade, new_period, name)
print(sqlUpd)

rp = const.execute(sqlUpd)
rp.rowcount


UPDATE buy
SET grade = "C2", period = "1" WHERE name = "TMT"



1

In [2]:
sql = '''
SELECT *
FROM buy
ORDER BY name'''
buy = pd.read_sql(sql, const)
buy['costAmt'] = buy.volbuy * buy.price
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy.dtypes

name                object
date        datetime64[ns]
shares               int64
price              float64
active               int64
dividend           float64
period              object
grade               object
costAmt            float64
dtype: object

In [3]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'costAmt':'{:,.2f}',    
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}',    
}

In [5]:
buy.style.format(format_dict)

,name,date,shares,price,active,dividend,period,grade,costAmt
0,ASP,2022-03-07,"30,000",3.80,1,0.4000,1,C4,"114,000.00"
1,BCH,2021-09-07,"21,000",20.90,1,1.2000,2,A3,"438,900.00"
2,CPNREIT,2022-08-16,"5,000",19.40,1,0.0000,2,A4,"97,000.00"
3,DCC,2022-05-05,"60,000",2.96,1,0.1900,2,B1,"177,600.00"
4,DIF,2020-08-01,"40,000",14.70,1,1.0430,2,B4,"588,000.00"
5,DOHOME,2021-08-20,"7,200",20.25,1,0.0000,4,B1,"145,800.00"
6,JASIF,2018-05-17,"130,000",10.00,1,0.9400,2,C4,"1,300,000.00"
7,KCE,2021-10-07,"15,000",71.75,1,1.6000,3,A4,"1,076,250.00"
8,MAKRO,2022-02-24,"7,500",40.40,1,0.7200,4,A4,"303,000.00"
9,MCS,2016-09-20,"75,000",15.40,1,1.2700,1,C3,"1,155,000.00"


In [7]:
sql = '''
SELECT name, year, quarter, q_eps, aq_eps, publish_date
FROM epss
WHERE year = 2022 AND quarter = 2'''
epss = pd.read_sql(sql, conlt)
epss.sample(5)

,name,year,quarter,q_eps,aq_eps,publish_date
109,IP,2022,2,0.10,0.20,2022-08-10
174,TVO,2022,2,1.07,1.82,2022-08-11
22,PTTEP,2022,2,5.28,7.92,2022-08-01
67,MBAX,2022,2,0.22,0.31,2022-08-10
91,SAT,2022,2,0.49,1.10,2022-08-10


In [8]:
df_merge = pd.merge(buy, epss, on='name', how='inner')
df_merge.sort_values(['period','name'],ascending=[True,True])

,name,date,shares,price,active,dividend,period,grade,costAmt,year,quarter,q_eps,aq_eps,publish_date
0,ASP,2022-03-07,30000,3.80,1,0.4000,1,C4,114000.0,2022,2,0.0200,0.0900,2022-07-27
9,MCS,2016-09-20,75000,15.40,1,1.2700,1,C3,1155000.0,2022,2,-0.1500,0.4100,2022-08-15
12,PTTGC,2021-03-17,12000,58.75,1,3.7500,1,C3,705000.0,2022,2,0.3100,1.2400,2022-08-10
17,STA,2021-06-15,22500,33.75,1,1.0000,1,C3,759375.0,2022,2,0.7400,1.7300,2022-08-10
1,BCH,2021-09-07,21000,20.90,1,1.2000,2,A3,438900.0,2022,2,0.4600,1.2700,2022-08-15
2,CPNREIT,2022-08-16,5000,19.40,1,0.0000,2,A4,97000.0,2022,2,0.3334,0.5147,2022-08-08
3,DCC,2022-05-05,60000,2.96,1,0.1900,2,B1,177600.0,2022,2,0.0470,0.1060,2022-08-09
4,DIF,2020-08-01,40000,14.70,1,1.0430,2,B4,588000.0,2022,2,0.2706,0.5429,2022-08-11
6,JASIF,2018-05-17,130000,10.00,1,0.9400,2,C4,1300000.0,2022,2,0.1765,0.3783,2022-08-09
13,RCL,2021-11-08,15000,45.75,1,6.0000,2,A1,686250.0,2022,2,8.8790,18.8000,2022-08-05


In [9]:
sql = '''
SELECT name, market
FROM stocks
ORDER BY name'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(252, 2)

In [10]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))    
]
values = ['SET50','SET100','mai']

In [11]:
my_stocks["mrkt"] = np.select(filters, values, default='SET')
my_stocks.head()

,name,market,mrkt
0,ACE,SET100,SET100
1,ADVANC,SET50 / SETHD / SETTHSI,SET50
2,AEONTS,SET100,SET100
3,AH,sSET / SETTHSI,SET
4,AIE,sSET,SET


In [12]:
df_merge2 = pd.merge(df_merge,my_stocks,on='name',how='inner')
df_merge2

,name,date,shares,price,active,dividend,period,grade,costAmt,year,quarter,q_eps,aq_eps,publish_date,market,mrkt
0,ASP,2022-03-07,30000,3.80,1,0.4000,1,C4,114000.0,2022,2,0.0200,0.0900,2022-07-27,sSET,SET
1,BCH,2021-09-07,21000,20.90,1,1.2000,2,A3,438900.0,2022,2,0.4600,1.2700,2022-08-15,SET100 / SETCLMV / SETHD / SETWB,SET100
2,CPNREIT,2022-08-16,5000,19.40,1,0.0000,2,A4,97000.0,2022,2,0.3334,0.5147,2022-08-08,SET,SET
3,DCC,2022-05-05,60000,2.96,1,0.1900,2,B1,177600.0,2022,2,0.0470,0.1060,2022-08-09,SET,SET
4,DIF,2020-08-01,40000,14.70,1,1.0430,2,B4,588000.0,2022,2,0.2706,0.5429,2022-08-11,SET,SET
5,DOHOME,2021-08-20,7200,20.25,1,0.0000,4,B1,145800.0,2022,2,0.1085,0.2666,2022-08-08,SET100 / SETWB,SET100
6,JASIF,2018-05-17,130000,10.00,1,0.9400,2,C4,1300000.0,2022,2,0.1765,0.3783,2022-08-09,SET,SET
7,KCE,2021-10-07,15000,71.75,1,1.6000,3,A4,1076250.0,2022,2,0.4800,0.9800,2022-08-09,SET50,SET50
8,MAKRO,2022-02-24,7500,40.40,1,0.7200,4,A4,303000.0,2022,2,0.1500,0.3400,2022-08-08,SET,SET
9,MCS,2016-09-20,75000,15.40,1,1.2700,1,C3,1155000.0,2022,2,-0.1500,0.4100,2022-08-15,sSET,SET


In [13]:
cols = 'period grade mrkt name shares price costAmt '.split()
df_merge2['shares'] = df_merge['shares'].astype(int)
df_merge2[cols].sort_values(['period', 'name'],ascending=[True,True]).style.format(format_dict)

,period,grade,mrkt,name,shares,price,costAmt
0,1,C4,SET,ASP,"30,000",3.80,"114,000.00"
9,1,C3,SET,MCS,"75,000",15.40,"1,155,000.00"
12,1,C3,SET50,PTTGC,"12,000",58.75,"705,000.00"
17,1,C3,SET100,STA,"22,500",33.75,"759,375.00"
1,2,A3,SET100,BCH,"21,000",20.90,"438,900.00"
2,2,A4,SET,CPNREIT,"5,000",19.40,"97,000.00"
3,2,B1,SET,DCC,"60,000",2.96,"177,600.00"
4,2,B4,SET,DIF,"40,000",14.70,"588,000.00"
6,2,C4,SET,JASIF,"130,000",10.00,"1,300,000.00"
13,2,A1,SET100,RCL,"15,000",45.75,"686,250.00"


In [13]:
df_merge2.groupby('period')['costAmt'].sum()

period
1    2733375.0
2    4435850.0
3    2807100.0
4    1363000.0
Name: costAmt, dtype: float64

### End of Quarterly Process

In [14]:
# look again at some of the data
catvars = ['period','grade']

for col in catvars:
  print(col, df_merge2[col].value_counts().sort_index(), sep="\n\n", end="\n\n\n")

period

1     4
2    10
3     6
4     5
Name: period, dtype: int64


grade

A1    1
A2    3
A3    2
A4    6
B1    2
B4    2
C1    1
C2    2
C3    4
C4    2
Name: grade, dtype: int64




In [15]:
df_merge2.loc[
    df_merge2.grade == 'A2',
['period','name','grade','costAmt']].sort_values(['period','name'],ascending=[True,True]).style.format(format_dict)

,period,name,grade,costAmt
21,2,TPIPL,A2,"102,000.00"
11,3,ORI,A2,"495,000.00"
20,3,TOP,A2,"172,500.00"


In [16]:
df_merge2.groupby('period').agg(
   {'costAmt':'sum','shares':'sum'}
).round(
   {'costAmt':2, 'shares':0}
).style.format(format_dict)

,costAmt,shares
period,,
1,"2,733,375.00","139,500"
2,"4,435,850.00","499,000"
3,"2,807,100.00","108,900"
4,"1,363,000.00","85,900"


In [19]:
df_merge2.replace(
   {
      'period':{'1':'Disposal','2':'Hi-Div','3':'Long Term','4':'Short Term'}
   }
).sample(5)

,name,date,shares,price,active,dividend,period,grade,costAmt,year,quarter,q_eps,aq_eps,publish_date,market,mrkt
19,TMT,2021-08-16,48000,9.90,1,1.3000,Hi-Div,C3,475200.0,2022,2,0.1900,0.3300,2022-08-11,sSET / SETTHSI,SET
8,MAKRO,2022-02-24,7500,40.40,1,0.7200,Short Term,A4,303000.0,2022,2,0.1500,0.3400,2022-08-08,SET,SET
24,WHART,2022-01-05,30000,11.70,1,0.7578,Short Term,C1,351000.0,2022,2,0.0419,0.2388,2022-08-08,SET,SET
12,PTTGC,2021-03-17,12000,58.75,1,3.7500,Disposal,C3,705000.0,2022,2,0.3100,1.2400,2022-08-10,SET50 / SETCLMV / SETTHSI,SET50
5,DOHOME,2021-08-20,7200,20.25,1,0.0000,Short Term,B1,145800.0,2022,2,0.1085,0.2666,2022-08-08,SET100 / SETWB,SET100


In [23]:
cols = 'period grade name shares price costAmt dividend q_eps aq_eps mrkt'.split()
df_merge2.replace(
   {
      'period':{'1':'Disposal','2':'Hi-Div','3':'Long Term','4':'Short Term'}
   }
)[cols].sort_values(['period','name'],ascending=[True,True]).style.format(format_dict)

,period,grade,name,shares,price,costAmt,dividend,q_eps,aq_eps,mrkt
0,Disposal,C4,ASP,"30,000",3.80,"114,000.00",0.4000,0.020000,0.090000,SET
9,Disposal,C3,MCS,"75,000",15.40,"1,155,000.00",1.2700,-0.150000,0.410000,SET
12,Disposal,C3,PTTGC,"12,000",58.75,"705,000.00",3.7500,0.310000,1.240000,SET50
17,Disposal,C3,STA,"22,500",33.75,"759,375.00",1.0000,0.740000,1.730000,SET100
1,Hi-Div,A3,BCH,"21,000",20.90,"438,900.00",1.2000,0.460000,1.270000,SET100
2,Hi-Div,A4,CPNREIT,"5,000",19.40,"97,000.00",0.0000,0.333400,0.514700,SET
3,Hi-Div,B1,DCC,"60,000",2.96,"177,600.00",0.1900,0.047000,0.106000,SET
4,Hi-Div,B4,DIF,"40,000",14.70,"588,000.00",1.0430,0.270600,0.542900,SET
6,Hi-Div,C4,JASIF,"130,000",10.00,"1,300,000.00",0.9400,0.176500,0.378300,SET
13,Hi-Div,A1,RCL,"15,000",45.75,"686,250.00",6.0000,8.879000,18.800000,SET100


### Start of change grade after quarterly eps

In [15]:
name = 'POPF'
sql = '''
SELECT * FROM buy
WHERE name = "%s"
'''
sql = sql % name
df = pd.read_sql(sql, con)
df

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,POPF,2021-07-30,10000.0,11.0,0.0,0.0,1,1.001,4,C1


In [14]:
sql = '''
UPDATE buy
SET period = "4", grade = "C1"
WHERE name = "%s"
'''
sql = sql % name
rp = con.execute(sql)
rp.rowcount

1

### End of change grade after quarterly eps

### Change period by name

In [29]:
name = 'MAKRO'
period = '3'
sqlUpd = '''
UPDATE buy
SET period = "%s"
WHERE name =  "%s"'''
sqlUpd = sqlUpd % (period, name)
print(sqlUpd)


UPDATE buy
SET period = "3"
WHERE name =  "MAKRO"


In [30]:
rp = const.execute(sqlUpd)
rp.rowcount

1

### End of Change period according to grade

### Change grade of specific stock

In [ ]:
sql = """
UPDATE buy
SET period = "2"
WHERE name IN ('ANAN','LH','MCS','ORI')"""
print(sql)

In [ ]:
sql = """
UPDATE buy
SET period = "3"
WHERE name IN ('AMATA')"""
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

### End of Change grade of specific stock

In [ ]:
sql = """
INSERT INTO buy
VALUES('GC','2018-02-28',10000,5.3,)"""

In [ ]:
sql = '''
UPDATE buy
SET period = "4"
WHERE grade <>  "A4"'''
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
sql = '''
UPDATE buy
SET period = "1"
WHERE substr(grade,1,1) <>  "A"'''
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
rp = con.execute(sql)
rp.rowcount

### Individual record change

In [ ]:
sql = """
UPDATE buy
SET period = "2"
WHERE name IN ('GC','LH','TISCO','MCS','BR')"""
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

### Par value change

In [ ]:
sql = """
SELECT *
FROM price
LIMIT 1"""
prices = pd.read_sql(sql, con)
prices.dtypes

In [ ]:
name = "PTT"
effective_date = "2018-04-24"

In [ ]:
sql = """
UPDATE price
SET price = price*10, maxp = maxp*10, minp = minp*10, opnp = opnp*10,
qty = qty/10
WHERE name = '%s' AND date < '%s'
"""
sql = sql % (name, effective_date)
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
sql = """
SELECT *
FROM buy
LIMIT 1"""
buys = pd.read_sql(sql, con)
buys.dtypes

In [ ]:
sql = """
UPDATE buy
SET volbuy = 4000, price = 50.8
WHERE name = '%s' AND active = 1
"""
sql = sql % (name)
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

### Name change

In [8]:
old_name = "EPCO"
new_name = "EP"

In [9]:
sql = """
SELECT *
FROM stockname
WHERE name = '%s'
"""
sql = sql % (old_name)
stock = pd.read_sql(sql, con)
stock

,name,category,sector,inport,onwatch,hd,top100
0,EPCO,,MEDIA,0,0,0,0


In [10]:
sql = """
UPDATE stockname
SET name = '%s'
WHERE name = '%s'
"""
sql = sql % (new_name, old_name)
print(sql)


UPDATE stockname
SET name = 'EP'
WHERE name = 'EPCO'



In [11]:
rp = con.execute(sql)
rp.rowcount

1

In [12]:
sql = """
SELECT *
FROM price
WHERE name = '%s'
"""
sql = sql % (old_name)
price = pd.read_sql(sql, con)
price.shape

(1533, 7)

In [13]:
sql = """
UPDATE price
SET name = '%s'
WHERE name = '%s'
"""
sql = sql % (new_name, old_name)
print(sql)


UPDATE price
SET name = 'EP'
WHERE name = 'EPCO'



In [14]:
rp = con.execute(sql)
rp.rowcount

1533

In [3]:
sql = '''
SELECT * FROM price WHERE name = 'ASK' ORDER BY date DESC'''
price = pd.read_sql(sql, con)
price

,name,date,price,maxp,minp,qty,opnp
0,ASK,2020-06-05,19.3,19.3,19.1,1180345,19.2
1,ASK,2020-06-04,19.1,19.4,19.0,984900,19.0
2,ASK,2020-06-02,18.8,19.1,18.7,488089,18.7
3,ASK,2020-06-01,18.7,18.9,18.5,1133238,18.5
4,ASK,2020-05-29,18.4,18.6,18.0,617730,18.3
5,ASK,2020-05-28,18.3,18.7,18.3,940551,18.5
6,ASK,2020-05-27,18.4,18.7,18.1,1498725,18.7
7,ASK,2020-05-26,18.4,18.5,18.0,1635709,18.0
8,ASK,2020-05-25,18.0,18.2,17.7,949532,17.9
9,ASK,2020-05-22,17.9,17.9,17.5,1657930,17.9


In [2]:
sql = '''
DELETE FROM price WHERE date = "2021-12-14"'''
rp = con.execute(sql)
rp.rowcount

0

In [11]:
sql = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sql)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [12]:
rcd = ['TSE', '2022-01-21', 40000.0, 2.60, 0, 0, 1, 0.10, '3', 'A4']
rcd

['TSE', '2022-01-21', 40000.0, 2.6, 0, 0, 1, 0.1, '3', 'A4']

In [15]:
rp = con.execute(sql, rcd)
rp.rowcount

1

In [20]:
sqlDel = '''
DELETE
FROM buy
WHERE name = "TSE"'''
rp = con.execute(sqlDel)
rp.rowcount

5

### SETIndex table

In [3]:
sql = '''
SELECT * FROM setindex WHERE setindex IS Null'''
df = pd.read_sql(sql, con)
df

,date,setindex
0,2022-04-27,None


In [4]:
sqlUpd = """
UPDATE setindex
SET setindex = 1661.89 WHERE setindex IS Null"""
print(sqlUpd)


UPDATE setindex
SET setindex = 1661.89 WHERE setindex IS Null


In [5]:
rp = con.execute(sqlUpd)
rp.rowcount

1

In [13]:
sqlIns = """
INSERT INTO setindex
VALUES('2022-04-25',1675.33)"""
print(sqlIns)


INSERT INTO setindex
VALUES('2022-04-25',1675.33)


In [14]:
rp = con.execute(sqlIns)
rp.rowcount

1